In [1]:
%pip install openai load_dotenv

  Using cached load_dotenv-0.1.0-py3-none-any.whl.metadata (1.9 kB)
  Using cached anyio-4.4.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached httpx-0.27.2-py3-none-any.whl.metadata (7.1 kB)
  Using cached jiter-0.5.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (3.6 kB)
  Using cached pydantic-2.9.1-py3-none-any.whl.metadata (146 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached tqdm-4.66.5-py3-none-any.whl.metadata (57 kB)
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
  Using cached certifi-2024.8.30-py3-none-any.whl.metadata (2.2 kB)
  Using cached httpcore-1.0.5-py3-none-any.whl.metadata (20 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic_core-2.23.3-cp312-cp312-macosx_11_0_arm64.whl.metadata (6.6 kB)
Using cached load_dotenv-0.1.0-py3-none-any.whl (7.2 k

In [2]:
# openai 패키지의 버전 정보 확인
%pip show openai | grep Version

Version: 1.45.0
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os 
from dotenv import load_dotenv
import openai
import json 
from openai import OpenAI


load_dotenv()


print(openai.__version__)

def show_json(obj):
    #obj의 모델을 Json 형태로 변환 후 출력
    display(json.loads(obj.model_dump_json()))


api_key = os.environ.get('OPENAI_API_KEY')
# Open ai API를 사용하기 위한 클라이언트 객체생성
client = OpenAI(api_key=api_key)


1.45.0


In [ ]:

assistant = client.beta.assistants.create(
    name = "상표 식별력 판단 AI(GPT-4o)",
    instructions="""
    1. Role
    상표 식별력 판단 AI 서비스는 업로드한 파일들을 Retrieval 하고, 상표의 이미지 또는 텍스트를 분석하여, 해당 상표의 식별력을 평가하고 등록 가능성을 판단하는 역할을 수행합니다.

    2. Context

    이 GPT는 대한민국의 상표를 새로 등록하려는 업체를 대상으로, 상표가 이미지든 텍스트든 상관없이 대한민국 상표심사기준 문서에 따라 식별력을 평가합니다. 또한, 식별력이 부족한 경우 그 이유를 상세히 설명하고, 관련 법률 조항과 판례를 참조하여 사용자가 이해할 수 있도록 도와줍니다.

    3. Dialog Flow

        1.	상표 입력 요청: 사용자가 상표 이미지를 업로드하거나 텍스트 형태의 상표를 입력하도록 요청합니다. 
        2.	상표 형태 인식: 입력된 상표가 이미지인지 텍스트인지 인식합니다.
        •	이미지일 경우: 이미지에서 텍스트나 시각적 요소를 추출한 후 분석합니다.
        •	텍스트일 경우: 텍스트를 직접 분석합니다.
        3.	법률 기준 참조: [상표심사기준202405.pdf]안에 있는 내용을 기준으로 상표의 식별력을 평가합니다.
        •	식별력이 충분한경우 :  GPT는 상표심사기준 문서에서 관련 조항을 참조하여, 왜 식별력이 있는지를 신뢰성있는 데이터를 바탕으로 상세히 설명합니다. 
        •	식별력이 부족한 경우: GPT는 상표심사기준 문서에서 관련 조항을 참조하여, 왜 식별력이 부족한지 이유를 신뢰성있는 데이터를 바탕으로 상세히 설명합니다.
        4.	결과 제공: 상표가 식별력이 있는지, 등록 가능성이 있는지 평가한 후 결과를 사용자에게 제공합니다. 이때, 식별력이 부족한 경우 해당 법률 조항을 명시하고 이유를 설명합니다.
        5.	피드백 요청: 사용자에게 제공된 분석 결과에 대해 피드백을 요청하고, 추가 분석이 필요한지 확인합니다.

    4. Instructions

        1.	상표를 이미지나 텍스트 형태로 입력받으세요.
        2.	상표의 형태를 인식한 후, 해당 상표를 분석하세요.
        •	이미지에서 텍스트를 추출하거나 시각적 요소를 분석합니다.
        •	텍스트 형태의 상표는 직접 분석합니다.
        3.	대한민국 [상표심사기준202405.pdf]에 따라 상표의 식별력을 평가하세요.
        •	식별력이 부족한 경우, 상표심사기준 문서에서 관련 조항을 참조하여 상세한 이유를 제공하세요.
        •	사용자가 신뢰를 느끼도록 참고한 문서의 메타데이터를 보여주세요
        4.	상표의 식별력 여부와 등록 가능성을 명확히 설명하고, 결과를 사용자에게 전달하세요.
        5.	사용자의 피드백을 받아 필요 시 추가 분석이나 법적 조언을 제공하세요.

    5. Constraints

        •	GPT는 상표의 이미지 또는 텍스트를 분석할 때 대한민국 상표심사기준에 따라 판단해야 합니다.
        •	식별력이 부족한 경우, 상표심사기준 문서의 관련 조항을 참조하여 상세한 이유를 제공해야 합니다.
        •	법률적 용어를 사용하여 명확하게 결과를 전달해야 합니다.

    6. Output Indicator

        •	상표가 식별력이 있는지 여부를 명확히 판단하고, 해당 상표가 등록 가능성이 있는지를 법률적 기준에 따라 정확히 평가합니다.
        •	식별력이 부족한 경우, 상표심사기준 문서에서 참조한 조항과 함께 구체적이고 명확한 이유를 제시합니다.
        •	사용자의 피드백에 따라 추가 분석이나 정보 제공이 이루어집니다.
""",
model='gpt-4o',

)
# 생성된 챗봇의 정보를 json 형태로 출력 
show_json(assistant)


In [5]:
assistant_delete = client.beta.assistants.delete(
    assistant_id = 'asst_x80ooPyJACW7qsswAChpQyMG'
)

In [4]:
# 기존 어시스턴트 ID 확인
assistant_list = client.beta.assistants.list()

for assistant in assistant_list:
    print(f"Assistant Name: {assistant.name}, Assistant ID: {assistant.id}")


Assistant Name: 상표 TEXT 유사도 평가 Assistant, Assistant ID: asst_IbYT7EatQkmSnremkg5RuiC0
Assistant Name: 상표 IMAGE 유사도 평가 Assistant, Assistant ID: asst_FY8Yfek8H3CrJKSLX2OyWFB1
Assistant Name: 수학천재, Assistant ID: asst_n01Ro5AKLNdP8Ye4IZqB727G
Assistant Name: 상표 식별력 판단 AI(GPT-4o), Assistant ID: asst_mD9MAguey0mzXs0wKEJmG4lV
Assistant Name: 상표 식별력 판단 AI, Assistant ID: asst_GDlNLfM4j2LCpTYFgULSR1V6


In [9]:
# assistant id를 별도의 변수에 담음
ASSISTANT_ID = 'asst_GDlNLfM4j2LCpTYFgULSR1V6'

print(f"[새로 생성한 ASSISTANT_ID]\n{ASSISTANT_ID}")


[새로 생성한 ASSISTANT_ID]
asst_GDlNLfM4j2LCpTYFgULSR1V6


In [5]:
# assistant 삭제
client.beta.assistants.delete(assistant_id='asst_MYUSHbmtVjpvB3wBQwFkmCp6')
# print(f"어시스턴트 {ASSISTANT_ID}가 삭제되었습니다.")

NotFoundError: Error code: 404 - {'error': {'message': "No assistant found with id 'asst_MYUSHbmtVjpvB3wBQwFkmCp6'.", 'type': 'invalid_request_error', 'param': None, 'code': None}}

# 업데이트 및 Code_interpreter
 assistant가 자체적으로 구동이 가능한 코드를 구현하고 query를 만들어 검색
- 데이터가 많을 경우 토큰을 효율적으로 사용가능

In [ ]:
# update assistant
assistant = client.beta.assistants.update(
    ASSISTANT_ID,
    tools=[{'type':'code_interpreter'}], 
)
show_json(assistant)

In [ ]:
def upload_files(files):
    uploaded_files = []
    for filepath in files:
        file = client.beta.create(
            file = open(
                filepath,
                'rb'
            ),
            purpose='assistants'
        )
        uploaded_files.append(file.id)
        print(f'[업로드한 파일 ID]\n{file.id}')
    return uploaded_files


In [15]:

#업로드할 파일들의 경로를 지정
files_to_uploaded = [
    'docs/[의견서예시][의견서미제출-거절]선행상표조사_MindShare.pdf',
    'docs/[의견서예시][의견서제출-거절]crople선행상표조사보고서(제출).pdf',
    'docs/[의견서예시]몸선필라테스&발레핏.pdf',
    'docs/상표검색 프로세스.pdf',
    'docs/상표심사기준202405.pdf',
    'docs/선행상표조사결과(샘플)_화음이 만든 샘플임_240822.pdf',
    'docs/상표유사여부보고서(별책).pdf',

]

file_streams = [open(path, 'rb') for path in files_to_uploaded]


### 파일 백터화

In [13]:
# vector_store = client.beta.vector_stores.create(
#     name = '상표 식별 documents',
# )

vector_store = client.beta.vector_stores.update(
    vector_store_id= 'vs_lnyjqbRPhkqR5RkQ3Y3pdiN1'
)
show_json(vector_store)

{'id': 'vs_lnyjqbRPhkqR5RkQ3Y3pdiN1',
 'created_at': 1725525376,
 'file_counts': {'cancelled': 0,
  'completed': 0,
  'failed': 0,
  'in_progress': 0,
  'total': 0},
 'last_active_at': 1725951855,
 'metadata': {},
 'name': '상표 식별 documents',
 'object': 'vector_store',
 'status': 'completed',
 'usage_bytes': 0,
 'expires_after': None,
 'expires_at': None}

In [11]:
client.beta.vector_stores.list()

SyncCursorPage[VectorStore](data=[VectorStore(id='vs_lnyjqbRPhkqR5RkQ3Y3pdiN1', created_at=1725525376, file_counts=FileCounts(cancelled=0, completed=3, failed=0, in_progress=0, total=3), last_active_at=1725601075, metadata={}, name='상표 식별 documents', object='vector_store', status='completed', usage_bytes=6974494, expires_after=None, expires_at=None), VectorStore(id='vs_8HRlMlOmpLEGKXIJAPZBGn2w', created_at=1725507210, file_counts=FileCounts(cancelled=0, completed=7, failed=0, in_progress=0, total=7), last_active_at=1725524851, metadata={}, name='상표 식별 documents', object='vector_store', status='completed', usage_bytes=15435659, expires_after=None, expires_at=None)], object='list', first_id='vs_lnyjqbRPhkqR5RkQ3Y3pdiN1', last_id='vs_8HRlMlOmpLEGKXIJAPZBGn2w', has_more=False)

In [53]:
client.files.delete(
    file_id=
        'file-hpjBTNyZRMpQG4lxlqGLaa2c'
    
)

FileDeleted(id='file-hpjBTNyZRMpQG4lxlqGLaa2c', deleted=True, object='file')

# Vector Store 삭제

In [3]:

## vectorstore 삭제 ###
vector_store = client.beta.vector_stores.delete(
    vector_store_id='vs_lnyjqbRPhkqR5RkQ3Y3pdiN1'
)

### 파일리스트보기

In [17]:
for file in client.files.list():
    print(f'[파일 ID] {file.id} [파일명]{file.filename}')

[파일 ID] file-XZ5MNOwKzsNqkKmdF554RSTI [파일명]상표유사여부보고서(별책).pdf
[파일 ID] file-bpwBjsH0mj1gV2TgNESh3Onf [파일명]상표심사기준202405.pdf
[파일 ID] file-qHMYWXfcBWdXu916aEL1YTZ3 [파일명][의견서예시]몸선필라테스&발레핏.pdf
[파일 ID] file-WVGkUZ1VYfv8RQowuemWRkV2 [파일명]선행상표조사결과(샘플)_화음이 만든 샘플임_240822.pdf
[파일 ID] file-R3VGc7hFw1t88RFsdqjGQwAM [파일명]상표검색 프로세스.pdf
[파일 ID] file-OM9q0naOczDAgVga6Y3S2HPM [파일명][의견서예시][의견서제출-거절]crople선행상표조사보고서(제출).pdf
[파일 ID] file-23Ztjs3hNvZtklt7w0Pge1bf [파일명][의견서예시][의견서미제출-거절]선행상표조사_MindShare.pdf
[파일 ID] file-LoNQ3oJJfZeU2sqLLPrW48Z7 [파일명]93215ee5-ecfa-461d-b52c-be0d54b760c7
[파일 ID] file-wr1HjPydhZO2YMmYD3D9nN1D [파일명]상표유사여부보고서(별책).pdf
[파일 ID] file-cZOWGW070kf1Zf8KrpaBlsYP [파일명]상표검색 프로세스.pdf
[파일 ID] file-X2Bolf8fllQW9jNeRTq3ccQq [파일명]상표심사기준202405.pdf
[파일 ID] file-HLvqqzo8tbEhAvF04olrbkyd [파일명]상표심사기준202405.pdf
[파일 ID] file-Egr1udFiMr7od8gyKqvzZ7cg [파일명]step_metrics.csv
[파일 ID] file-yWxyD77DLvXAki3qebW1H7Vy [파일명]test2.jsonl
[파일 ID] file-VRH4cOFuzVCZx2vbEv8K57b4 [파일명]step_metrics.csv
[파일 ID] file-xIobt8B

### 2. 파일을 업로드 하고 vector store에 추가

In [16]:
# 파일 업로드 및 백터 스토어에 추가
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
    vector_store_id=vector_store.id, files = file_streams
)

In [18]:
show_json(file_batch)
print(file_batch.status)
print(file_batch.file_counts)

{'id': 'vsfb_f588f0705ce643819c9de61e6acee60c',
 'created_at': 1725951996,
 'file_counts': {'cancelled': 0,
  'completed': 7,
  'failed': 0,
  'in_progress': 0,
  'total': 7},
 'object': 'vector_store.file_batch',
 'status': 'completed',
 'vector_store_id': 'vs_lnyjqbRPhkqR5RkQ3Y3pdiN1'}

completed
FileCounts(cancelled=0, completed=7, failed=0, in_progress=0, total=7)


In [ ]:
#assistant 설정 업데이트
assistant = client.beta.assistants.update(
    ASSISTANT_ID,
    tools=[
        {'type': 'code_interpreter'},
        {'type': "file_search"}
    ]
)

### 3. 새 백터 스토어를 사용하도록 어시스턴트 업데이트

In [ ]:
# 업로드된 파일 참조 및 검색 요청
assistant = client.beta.assistants.update(
    assistant_id=ASSISTANT_ID,
    tool_resources={'file_search': {'vector_store_ids': [vector_store.id]}},
    temperature=0
)
show_json(assistant)

### 4. 스레드 만들기
스레드에 메시지 첨부파일로 파일을 첨부 가능
vector_store 스레드와 연관된 다른 파일이 생성되거나, 스레드에 이미 벡터 스토어가 첨부 되어있는 경우 새 파일을 기존 스레드 벡터 스토어에 첨부


	1.	파일 업로드: message_file 객체는 업로드된 이미지의 파일 ID를 가지고 있으며, 이 파일 ID를 스레드에서 사용하여 이미지 파일을 참조합니다.
	2.	스레드 생성: thread 객체는 대화의 흐름을 관리하는 스레드이며, 여기에 사용자가 보낸 메시지(텍스트와 이미지)가 포함됩니다.
	3.	어시스턴트 실행: client.beta.threads.runs.create()를 사용하여 스레드 내에서 어시스턴트를 실행시킵니다. 이때 어시스턴트의 ID가 필요하며, 해당 어시스턴트가 이미지를 분석할 수 있는지 확인해야 합니다.
	4.	결과 확인: client.beta.threads.messages.list()로 스레드의 대화 내용을 확인하여 어시스턴트가 이미지에 대해 어떤 응답을 했는지 확인할 수 있습니다.


In [36]:
thread = client.beta.threads.create()

show_json(thread)


{'id': 'thread_jJe5jeKUpNGHGVHMUah0mZ2v',
 'created_at': 1725966173,
 'metadata': {},
 'object': 'thread',
 'tool_resources': {'code_interpreter': None, 'file_search': None}}

In [37]:
message = client.beta.threads.messages.create(
    thread_id= thread.id,
    role='user',
    content='식별력이 뭐야?'
)
show_json(message)

{'id': 'msg_eXjiNWLrjmjJZOZAHMVNBO0N',
 'assistant_id': None,
 'attachments': [],
 'completed_at': None,
 'content': [{'text': {'annotations': [], 'value': '식별력이 뭐야?'},
   'type': 'text'}],
 'created_at': 1725966182,
 'incomplete_at': None,
 'incomplete_details': None,
 'metadata': {},
 'object': 'thread.message',
 'role': 'user',
 'run_id': None,
 'status': None,
 'thread_id': 'thread_jJe5jeKUpNGHGVHMUah0mZ2v'}

In [38]:
run = client.beta.threads.runs.create(
    thread_id=thread.id, # 생성한스레드(카톡방)
    assistant_id=ASSISTANT_ID # 적용할 AssistantID
)
show_json(run)

{'id': 'run_J2zh5D9zYCZH1hV6AmCdkKPd',
 'assistant_id': 'asst_GDlNLfM4j2LCpTYFgULSR1V6',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1725966186,
 'expires_at': 1725966786,
 'failed_at': None,
 'incomplete_details': None,
 'instructions': '\n당신은 상표 식별력, 유사도를 판단하여 의견서를 제시해주는 AI 입니다.\n아래와 같은 instructions를 따라 문서를 검색하고, 업로드 되어있는 의견서 예시와 같은 형식으로 대답하세요.\n\n1. Role\n\n상표 식별력 판단 AI 서비스는 [상표심사기준202405.pdf]를 기준으로, 상표의 이미지 또는 텍스트를 분석하여, 해당 상표의 식별력을 평가하고 등록 가능성을 판단하는 역할을 수행합니다.\n\n2. Context\n\n이 GPT는 대한민국의 상표를 새로 등록하려는 업체를 대상으로, 상표가 이미지든 텍스트든 상관없이 대한민국 상표심사기준 문서에 따라 식별력을 평가합니다. 또한, 식별력이 부족한 경우 그 이유를 상세히 설명하고, 관련 법률 조항과 판례를 참조하여 사용자가 이해할 수 있도록 도와줍니다.\n\n3. Instructions\n\n\t1.\t상표 입력 요청: 사용자가 상표 이미지를 업로드하거나 텍스트 형태의 상표를 입력하도록 요청합니다.\n\t2.\t상표 형태 인식: 입력된 상표가 이미지인지 텍스트인지 인식합니다.\n\t    - 이미지일 경우: 이미지에서 텍스트나 시각적 요소를 추출한 후 분석합니다.\n\t    - 텍스트일 경우: 텍스트를 직접 분석합니다.\n\t3.\t[상표 검색 프로레스.pdf]를 따라 해당 의견서를 작성합니다.\n\t4.\t결과 제공: [상표심사기준202405.pdf]의 법률 기준을 참조하여 상표의 식별력 및 유사성을 평가 후 의견서를 제시하세요. 반드시 업로

run을 생성하는 것은 비동기 작업이다.
이는 Run의 메타데이터와 함께 즉시 반환되며, status 는 queued(대기중)으로 표기된다.
status 는 Assistant가 작업을 수행함에 따라(도구 사용 및 메시지 추가와 같은) 업데이트될 것임.

### status 목록
- queued : 아직 실행이 되지 않고 대기중인 상태
- in_progress: 처리중
- requires_action : 사용자 입력 대기중
- cancelling : 작업 취소중
- cancelled : 취소 완료
- failed: 오류(실패)
- completed: 작업완료
- expired: 작업 만료

In [39]:
import time 

def wait_on_run(run, thread):
    # 주어진 실행 (run)이 완료 될때까지 대기
    # status 가 'queued' 또는 'inprogress'인 경우에는 계속 polling 하며 대기
    while run.status == 'queued' or run.status == 'in_progress':
        # run.status를 업데이트 합니다.
        run = client.beta.threads.runs.retrieve(
            thread_id = thread.id,
            run_id= run.id
        )
        time.sleep(0.5)
    return run

In [40]:
# run 객체를 대기 상태로 설정하고, 해당 스레드에서 실행을 완료할 때가지 기다림
run = wait_on_run(run, thread)

# status가 'complete'인 경우에는 결과를 출력합니다.
show_json(run)

{'id': 'run_J2zh5D9zYCZH1hV6AmCdkKPd',
 'assistant_id': 'asst_GDlNLfM4j2LCpTYFgULSR1V6',
 'cancelled_at': None,
 'completed_at': 1725966189,
 'created_at': 1725966186,
 'expires_at': None,
 'failed_at': None,
 'incomplete_details': None,
 'instructions': '\n당신은 상표 식별력, 유사도를 판단하여 의견서를 제시해주는 AI 입니다.\n아래와 같은 instructions를 따라 문서를 검색하고, 업로드 되어있는 의견서 예시와 같은 형식으로 대답하세요.\n\n1. Role\n\n상표 식별력 판단 AI 서비스는 [상표심사기준202405.pdf]를 기준으로, 상표의 이미지 또는 텍스트를 분석하여, 해당 상표의 식별력을 평가하고 등록 가능성을 판단하는 역할을 수행합니다.\n\n2. Context\n\n이 GPT는 대한민국의 상표를 새로 등록하려는 업체를 대상으로, 상표가 이미지든 텍스트든 상관없이 대한민국 상표심사기준 문서에 따라 식별력을 평가합니다. 또한, 식별력이 부족한 경우 그 이유를 상세히 설명하고, 관련 법률 조항과 판례를 참조하여 사용자가 이해할 수 있도록 도와줍니다.\n\n3. Instructions\n\n\t1.\t상표 입력 요청: 사용자가 상표 이미지를 업로드하거나 텍스트 형태의 상표를 입력하도록 요청합니다.\n\t2.\t상표 형태 인식: 입력된 상표가 이미지인지 텍스트인지 인식합니다.\n\t    - 이미지일 경우: 이미지에서 텍스트나 시각적 요소를 추출한 후 분석합니다.\n\t    - 텍스트일 경우: 텍스트를 직접 분석합니다.\n\t3.\t[상표 검색 프로레스.pdf]를 따라 해당 의견서를 작성합니다.\n\t4.\t결과 제공: [상표심사기준202405.pdf]의 법률 기준을 참조하여 상표의 식별력 및 유사성을 평가 후 의견서를 제시하세요. 반드시 업로

### Message(메세지)

run이 완료되었다면, Assistant에 의해 처리된 결과를 보기 위해 Thread에서 messages를 확인할 수 있다.

In [41]:
# thread.id 를 사용하여 메시지 목록을 가져옴
messages = client.beta.threads.messages.list(thread_id=thread.id)

#결과 출력
show_json(messages)

{'data': [{'id': 'msg_IYW6ZSBRUpQfke693rtYeeHd',
   'assistant_id': 'asst_GDlNLfM4j2LCpTYFgULSR1V6',
   'attachments': [],
   'completed_at': None,
   'content': [{'text': {'annotations': [],
      'value': '식별력은 상표가 특정 상품이나 서비스의 출처를 식별할 수 있는 능력을 의미합니다. 즉, 소비자가 해당 상표를 보고 특정 기업이나 제품을 인식할 수 있는 정도를 나타냅니다. 식별력이 높은 상표는 소비자에게 강한 인상을 주고, 다른 상표와 쉽게 구별될 수 있습니다.\n\n상표의 식별력은 일반적으로 다음과 같은 기준에 따라 평가됩니다:\n\n1. **고유성**: 상표가 얼마나 독창적이고 고유한지를 평가합니다. 일반적인 용어, 설명적인 표현, 또는 통상적인 기호는 식별력이 낮습니다.\n\n2. **사용의 연속성**: 상표가 시장에서 얼마나 오랫동안 사용되었는지, 그리고 소비자에게 얼마나 잘 알려져 있는지를 고려합니다.\n\n3. **소비자의 인식**: 소비자가 해당 상표를 보고 어떤 상품이나 서비스를 떠올리는지를 평가합니다.\n\n식별력이 부족한 상표는 상표 등록이 거부될 수 있으며, 이는 상표법에 따라 보호받지 못할 수 있습니다.'},
     'type': 'text'}],
   'created_at': 1725966187,
   'incomplete_at': None,
   'incomplete_details': None,
   'metadata': {},
   'object': 'thread.message',
   'role': 'assistant',
   'run_id': 'run_J2zh5D9zYCZH1hV6AmCdkKPd',
   'status': None,
   'thread_id': 'thread_jJe5jeKUpNGHGVHMUah0mZ2v'},
  {'id': 'msg_eXjiNWLr

이전에 받은 답변을 기억하고 있는지 확인

In [42]:
message = client.beta.threads.messages.create(
    thread_id= thread.id,
    role='user',
    content='더 간단하게 말해줄래?'
)

run = client.beta.threads.runs.create(
    thread_id = thread.id,
    assistant_id= ASSISTANT_ID
)

# 답변 완료될때까지 대기
wait_on_run(run, thread)

# 마지막 사용자 메시지 이후 추가된 모든 메시지 검색
messages = client.beta.threads.messages.list(
    thread_id=thread.id, 
    order='asc', #오름차순으로 출력
    after=message.id  # 이전 메세지를 제외하고 출력
)
show_json(messages)

{'data': [{'id': 'msg_w3H09hSN0pd6b7jmQc2rx8Ee',
   'assistant_id': 'asst_GDlNLfM4j2LCpTYFgULSR1V6',
   'attachments': [],
   'completed_at': None,
   'content': [{'text': {'annotations': [],
      'value': '식별력은 상표가 특정 제품이나 서비스의 출처를 알아볼 수 있는 능력을 말해요. 쉽게 말해, 소비자가 그 상표를 보고 어떤 회사나 제품인지 바로 알 수 있는 정도입니다. 식별력이 높으면 소비자에게 잘 알려져 있고, 다른 상표와 쉽게 구별할 수 있어요. 반대로, 식별력이 낮으면 상표 등록이 어려울 수 있습니다.'},
     'type': 'text'}],
   'created_at': 1725966231,
   'incomplete_at': None,
   'incomplete_details': None,
   'metadata': {},
   'object': 'thread.message',
   'role': 'assistant',
   'run_id': 'run_Zt85nwsY9QxvSJZvlIwGLsX6',
   'status': None,
   'thread_id': 'thread_jJe5jeKUpNGHGVHMUah0mZ2v'}],
 'object': 'list',
 'first_id': 'msg_w3H09hSN0pd6b7jmQc2rx8Ee',
 'last_id': 'msg_w3H09hSN0pd6b7jmQc2rx8Ee',
 'has_more': False}

아미지 업로드와 함께 물어보기.

In [44]:
file = client.files.create(
    file = open('brand_img/여기의성.png','rb'),
    purpose='vision'
)

message = client.beta.threads.messages.create(
    thread_id= thread.id,
    role='user',
    content=[
        {
            'type': 'text',
            'text': '이 상표의 의견서를 제시해주세요'
        },
        {
            'type': 'image_file',
            'image_file': {'file_id': file.id}
        }
    ],
    
)

run = client.beta.threads.runs.create(
    thread_id = thread.id,
    assistant_id= ASSISTANT_ID,
    tools=[]
)

# 답변 완료될때까지 대기
wait_on_run(run, thread)

# 마지막 사용자 메시지 이후 추가된 모든 메시지 검색
messages = client.beta.threads.messages.list(
    thread_id=thread.id, 
    # order='asc', #오름차순으로 출력
    # after=message.id  # 이전 메세지를 제외하고 출력
)
show_json(messages)

{'data': [{'id': 'msg_USEGenAv6pKmEfNnOXCwr75F',
   'assistant_id': 'asst_GDlNLfM4j2LCpTYFgULSR1V6',
   'attachments': [],
   'completed_at': None,
   'content': [{'text': {'annotations': [],
      'value': '상표 "여기 의성"에 대한 의견서를 아래와 같이 제시합니다.\n\n---\n\n**조사 대상 상표**: 여기 의성\n\n**조사 결과 및 검토의견**: \n(1) 선행상표 조사결과: \n- "여기"라는 단어는 일반적인 위치를 나타내는 표현으로, 여러 상표에서 사용될 수 있는 용어입니다.\n- "의성"은 특정 지명을 나타내며, 지역명으로서의 기능을 가지고 있습니다.\n- 따라서, "여기 의성"이라는 조합은 특정 지역을 지칭하는 설명적인 의미를 가집니다.\n\n(2) 검토의견: \n- "여기 의성"은 특정 지역을 나타내는 표현으로, 소비자에게 특정 상품이나 서비스의 출처를 명확히 식별할 수 있는 능력이 부족합니다. \n- 일반적인 지명과 위치를 나타내는 표현의 조합으로 인해, 다른 유사한 상표와의 혼동 가능성이 높습니다.\n\n**식별력(적합성)**: \n- [상표심사기준202405.pdf]에 따르면, 상표가 일반적인 용어 또는 설명적인 표현일 경우 식별력이 부족하다고 판단됩니다. \n- "여기 의성"은 특정 지역을 설명하는 용어로, 소비자가 이를 보고 특정 기업이나 제품을 떠올리기 어려운 점에서 식별력이 낮다고 평가됩니다.\n\n**종합의견(유사성 및 기타의견)**: \n- "여기 의성"은 지역명과 일반적인 표현의 조합으로, 식별력이 부족하여 상표 등록이 어려울 것으로 보입니다. \n- 유사한 상표가 존재할 가능성이 높으며, 소비자에게 혼동을 줄 수 있는 점에서 등록 가능성이 낮습니다.\n\n---\n\n이 의견서에 대한 피드백이나 추가 분석이 필요하신 경우 말씀해 주세요.'},
     'typ

In [59]:

def submit_message(assistant_id, thread, user_message):
    #사용자 입력 메시지를 스레드에 추가
    client.beta.threads.messages.create(
        thread_id= thread.id,
        role = "user",
        content = user_message
    )

    #스레드에 메시지가 입력되었다면 실행 준비
    run= client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant_id
    )

    return run

def wait_on_run(run,thread):
    # run이 완료될대까지 기다림 : polling 하며 대기 (polling: 서버와 응답을 주고받음)
    while run.status == 'queued' or run.status == 'in_progress':
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id
        )
        time.sleep(0.5)
    return run

def get_response(thread):
    # 스레드에서 메세지 목록가져오기
    return client.beta.threads.messages.list(thread_id=thread.id, order='asc')


# 새로운 스레드 생성 및 메시지 제출 함수
def create_thread_and_run(user_input):
    # 사용자 입력을 받아 새로운 스래드를 생성하고, Assistant 에게 메시지를 제출
    thread= client.beta.threads.create()
    run = submit_message(ASSISTANT_ID, thread, user_input)
    return thread, run


In [65]:
# 동시에 여러 요청을 처리하기 위해 스래드를 생성합니다.
thread1, run1 = create_thread_and_run('스타벅스 상표의 식별력은 어떤가요?')
thread2, run2 = create_thread_and_run('어떤 기준이 가장 중요한가요?')
thread3, run3 = create_thread_and_run('어떻게하면 식별력 있는 상표를 만들 수 있을까요?')

In [66]:
import time  
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
# 메시지 출력용 함수
def print_message(response):
    for res in response:
        print(f'[{res.role.upper()}]\n{res.content[0].text.value}\n')
    print("-" * 60)

#반복문에서 대기하는 함수
def wait_on_run(run, thread):
    while run.status == 'queued' or run.status == 'in_progress':
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id = run.id
        )
        time.sleep(0.5)
    return run




# 첫 번째 실행을 위해 대기
run1 = wait_on_run(run1, thread1)
print_message(get_response(thread1))

# 두 번째 실행을 위해 대기
run2 = wait_on_run(run2, thread2)
print_message(get_response(thread2))

# 세 번째 실행을 위해 대기
run3 = wait_on_run(run3, thread3)
# 세 번째 스레드를 마치면 감사 인사를 전하고 종료합니다 :)
run4 = submit_message(ASSISTANT_ID, thread3, "도와주셔서 감사합니다!")
run4 = wait_on_run(run4, thread3)
print_message(get_response(thread3))


[USER]
스타벅스 상표의 식별력은 어떤가요?

[ASSISTANT]
스타벅스 상표에 대한 식별력을 평가하기 위해, 먼저 상표의 이미지나 텍스트를 제공해 주시기 바랍니다. 상표가 이미지인 경우 이미지를 업로드해 주시고, 텍스트인 경우 상표명을 입력해 주시면 분석을 진행하겠습니다.

------------------------------------------------------------
[USER]
어떤 기준이 가장 중요한가요?

[ASSISTANT]
상표의 식별력 판단에 있어 가장 중요한 기준은 다음과 같습니다:

1. **식별력 유무에 대한 심사**: 상표가 특정 상품이나 서비스를 식별할 수 있는 능력을 평가합니다. 이는 상표가 소비자에게 그 출처를 명확히 인식시킬 수 있는지를 판단하는 데 중점을 둡니다【4:3†source】.

2. **유사여부에 대한 심사**: 등록된 상표와의 유사성을 평가하여, 소비자가 혼동할 가능성이 있는지를 판단합니다. 이는 상표의 독창성과 차별성을 확보하는 데 중요한 요소입니다【4:19†source】.

3. **기능성 유무에 대한 심사**: 상표가 기능적인 요소를 포함하고 있는지를 평가합니다. 기능성이 인정되는 경우, 상표 등록이 거부될 수 있습니다【4:4†source】.

이 외에도 상표의 유형, 사용에 의한 식별력, 그리고 법적 요건 등이 중요한 기준으로 작용합니다. 상표의 등록 가능성을 높이기 위해서는 이러한 기준들을 충족하는 것이 필수적입니다.

------------------------------------------------------------
[USER]
어떻게하면 식별력 있는 상표를 만들 수 있을까요?

[ASSISTANT]
식별력 있는 상표를 만들기 위해서는 다음과 같은 몇 가지 원칙을 고려해야 합니다:

1. **독창성**: 상표는 다른 상표와 구별될 수 있도록 독창적이어야 합니다. 일반적인 단어, 문구 또는 디자인을 피하고, 고유한 요소를 포함하는 것이 좋습니다.

2. **기술적 설명 피하기**: 